In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
import checklist
from checklist.editor import Editor
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper
from checklist.test_types import MFT
from typing import List
import warnings
warnings.filterwarnings('ignore')

# MFTs: Introduction
In this notebook, we will create Minimum Functionality Tests (MFTs) for a generative language model. MFTs test one specific function of a language model. They are analogous to unit tests in traditional software engineering.

## Setup generative model
Before we can test anything, we need to set up our language model. We will use the HuggingFace transformers library to load a GPT2 model.

First, we create a tokenizer. The tokenizer is responsible for splitting strings into individual words, then converting those words into vectors of numbers that our model can understand.

In [2]:
# Load pretrained model tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Demonstrate what the tokenizer does
tokenizer.encode("Wherefore art thou Romeo?")

[8496, 754, 1242, 14210, 43989, 30]

Our tokenizer has turned the human-readable text into a list of numbers that the model understands. Next, let's load the GPT2 model.

In [3]:
# Load pretrained model (weights)
model = GPT2LMHeadModel.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id)
device = 'cuda'
model.eval()
model.to(device)
"Model loaded"

'Model loaded'

Generating text with the model requires a bit of work. Let's write a function `generate_sentences` to handle the text generation.

`generate_sentences` has 1 parameter, `prompts`, which is a list of strings. A prompt is a string that the model will use as a starting point for generating new text. It gives the model context about what kind of text should be generated.

`generate_sentences` will output a list of generated text responses for each prompt.

In [4]:
def generate_sentences(prompts: List[str]) -> str:
    sentences = []
    for prompt in prompts:
        token_tensor = tokenizer.encode(prompt, return_tensors='pt').to(device) # return_tensors = "pt" returns a PyTorch tensor
        out = model.generate(
            token_tensor,
            do_sample=True,
            min_length=10,
            max_length=50,
            num_beams=1,
            no_repeat_ngram_size=2,
            early_stopping=False,
            output_scores=True,
            return_dict_in_generate=True)
        text = tokenizer.decode(out.sequences[0], skip_special_tokens=True)
        sentences.append(text[len(prompt):])
    return sentences

In [5]:
generate_sentences(["Wherefore art thou Romeo?"])

[' What art ye here, then?!\n\n\nAlarubaeus: What, but what thou art saying, to what a fool, sayest not, thou poor fool! and for the fool what is thou? I']

Now that everything is ready, we can write our first MFT.

## MFT - Language prompt
For this MFT, we will expect the model to create a reasonable continuation of a prompt. The model will be prompted with strings like "In {country} the most commonly spoken language is " where {country} is a placeholder for a country such as Spain.

We need create a rule to determine if the model passes our test. The criteria for passing or failing the test is entirely user defined. We will consider this MFT to pass if the model's output contains any language name. This will demonstrate that the model understands the general context of the prompt. The mentioned language doesn't have to be accurate - for example, "In Spain the most commonly spoken language is Indonesian" would pass our test, because Indonesian is a language. The language may also be located anywhere in the output - for example, "In Spain the most commonly spoken language is not easy to learn. Spanish has many complicated conjugations." would also pass our test.

In a later section of this notebook, there is another version of this MFT that is stricter, requiring the correct language to be mentioned in the response.

### Handwritten MFT
First, we will write the MFT by hand. Then, we'll use Checklist's MFT class to demonstrate how Checklist helps us create the MFT much more quickly.

#### Generate prompts from template
We will use Checklist's Editor class to quickly create the prompts.

In [6]:
editor = Editor()
prompt_strs = editor.template("In {country} the most commonly spoken language is ", nsamples=10)
prompt_strs.data

['In Uzbekistan the most commonly spoken language is ',
 'In Greece the most commonly spoken language is ',
 'In Cameroon the most commonly spoken language is ',
 'In Palau the most commonly spoken language is ',
 'In Orange Free State the most commonly spoken language is ',
 'In Tajikistan the most commonly spoken language is ',
 'In Mozambique the most commonly spoken language is ',
 'In Qatar the most commonly spoken language is ',
 'In Tonga the most commonly spoken language is ',
 'In Qatar the most commonly spoken language is ']

#### Language CSV
We need a list of languages to check if the model's output contains a language. To save some time, we will read language names from a CSV file. The data comes from standard ISO Language Codes https://datahub.io/core/language-codes 

In [7]:
import urllib.request
urllib.request.urlretrieve('https://datahub.io/core/language-codes/r/language-codes.csv', 'language-codes.csv')
lang_codes_csv = pd.read_csv('language-codes.csv')
lang_codes_csv

,alpha2,English
0,aa,Afar
1,ab,Abkhazian
2,ae,Avestan
3,af,Afrikaans
4,ak,Akan
...,...,...
179,yi,Yiddish
180,yo,Yoruba
181,za,Zhuang; Chuang
182,zh,Chinese


#### Run the MFT
Now we're ready to create the MFT. We will create 3 Pandas dataframes, one each for prompts, responses, and results. Then, we will loop over the prompts, send each prompt to the model, and determine if it passes or fails the test. Each prompt and its test result will be recorded in the dataframes.

In [8]:
prompts = pd.DataFrame({"id": [], "prompt": []})
responses = pd.DataFrame({"id": [], "response": []})
results = pd.DataFrame({"id": [], "p/f": []})
langs = lang_codes_csv["English"].tolist()

model_responses = generate_sentences(prompt_strs.data)

for (i, response) in enumerate(model_responses):
    pf = 'fail'
    
    # Check if any language from the CSV data is in the generated string
    for l in langs:
        if l in response:
            pf = 'pass'
            break

    prompts = prompts.append({"id": i, "prompt": prompt_strs.data[i]}, ignore_index=True)
    responses = responses.append({"id": i, "response": response}, ignore_index=True)
    results = results.append({"id": i, "p/f": pf}, ignore_index=True)

#### Show test results
Now let's look at the results of our test.

In [9]:
pd.set_option("max_colwidth", 250)

In [10]:
prompts

,id,prompt
0,0.0,In Uzbekistan the most commonly spoken language is
1,1.0,In Greece the most commonly spoken language is
2,2.0,In Cameroon the most commonly spoken language is
3,3.0,In Palau the most commonly spoken language is
4,4.0,In Orange Free State the most commonly spoken language is
5,5.0,In Tajikistan the most commonly spoken language is
6,6.0,In Mozambique the most commonly spoken language is
7,7.0,In Qatar the most commonly spoken language is
8,8.0,In Tonga the most commonly spoken language is
9,9.0,In Qatar the most commonly spoken language is


In [11]:
responses

,id,response
0,0.0,"vernacular in Uzbek. In Uzbek, the language of learning is ""Zarman,"" and it is usually given as ң.\n\nThe main differences on the pronunciation of ""zar"
1,1.0,"합국어 (에비지성한), a variant of it written with the same spelling.\n\nIn Russia the term �"
2,2.0,"ʻuhb-bah and it is called ukka-bah or 'titum-ah', while Arabic and Latin are the two most common.\n\nMany people use the term"
3,3.0,"vernacular, especially Mandarin. In the modern world, it's often used in the Western sense, to describe things as they would today. It's also considered part of the language barrier because of its"
4,4.0,㆜ 앴문 (koreansu) and 不古는 (junku). 佧검語 (lun
5,5.0,"ʇḍāni. This language, which means English, is a popular means for communication in and around Tajigistan. It is the only form of Arabic spoken in Tajand in"
6,6.0,"vernacular, though with less use in the Western Sahara. In the western Sahara, Ṭūc is commonly used in French with emphasis on it being a sign of peace.\n\n"
7,7.0,"ā, of which English is almost as common as Arabic and Urdu. Arabic-derived languages, in contrast, also have a relatively lower rate of usage than English. For example, Urati is about"
8,8.0,əsak in the Tonganyat language (Pinnayatat). It's also said to have a strong dialect with some similarities between the languages. Tongans are also referred to as
9,9.0,"اجمبیر Arabic means the name for Mecca. Other than these two words it is the same Arabic language. It also means ""the best and most famous city in the world"".\n"


In [12]:
results

,id,p/f
0,0.0,pass
1,1.0,fail
2,2.0,pass
3,3.0,fail
4,4.0,fail
5,5.0,pass
6,6.0,pass
7,7.0,pass
8,8.0,fail
9,9.0,pass


We can merge all the dataframes to make the results easier to read.

In [13]:
merged = pd.merge(responses, results, on="id")
merged = pd.merge(prompts, merged, on="id")
merged

,id,prompt,response,p/f
0,0.0,In Uzbekistan the most commonly spoken language is,"vernacular in Uzbek. In Uzbek, the language of learning is ""Zarman,"" and it is usually given as ң.\n\nThe main differences on the pronunciation of ""zar",pass
1,1.0,In Greece the most commonly spoken language is,"합국어 (에비지성한), a variant of it written with the same spelling.\n\nIn Russia the term �",fail
2,2.0,In Cameroon the most commonly spoken language is,"ʻuhb-bah and it is called ukka-bah or 'titum-ah', while Arabic and Latin are the two most common.\n\nMany people use the term",pass
3,3.0,In Palau the most commonly spoken language is,"vernacular, especially Mandarin. In the modern world, it's often used in the Western sense, to describe things as they would today. It's also considered part of the language barrier because of its",fail
4,4.0,In Orange Free State the most commonly spoken language is,㆜ 앴문 (koreansu) and 不古는 (junku). 佧검語 (lun,fail
5,5.0,In Tajikistan the most commonly spoken language is,"ʇḍāni. This language, which means English, is a popular means for communication in and around Tajigistan. It is the only form of Arabic spoken in Tajand in",pass
6,6.0,In Mozambique the most commonly spoken language is,"vernacular, though with less use in the Western Sahara. In the western Sahara, Ṭūc is commonly used in French with emphasis on it being a sign of peace.\n\n",pass
7,7.0,In Qatar the most commonly spoken language is,"ā, of which English is almost as common as Arabic and Urdu. Arabic-derived languages, in contrast, also have a relatively lower rate of usage than English. For example, Urati is about",pass
8,8.0,In Tonga the most commonly spoken language is,əsak in the Tonganyat language (Pinnayatat). It's also said to have a strong dialect with some similarities between the languages. Tongans are also referred to as,fail
9,9.0,In Qatar the most commonly spoken language is,"اجمبیر Arabic means the name for Mecca. Other than these two words it is the same Arabic language. It also means ""the best and most famous city in the world"".\n",pass


Finally, let's display the failing tests.

In [14]:
merged.loc[merged['p/f'] == 'fail']

,id,prompt,response,p/f
1,1.0,In Greece the most commonly spoken language is,"합국어 (에비지성한), a variant of it written with the same spelling.\n\nIn Russia the term �",fail
3,3.0,In Palau the most commonly spoken language is,"vernacular, especially Mandarin. In the modern world, it's often used in the Western sense, to describe things as they would today. It's also considered part of the language barrier because of its",fail
4,4.0,In Orange Free State the most commonly spoken language is,㆜ 앴문 (koreansu) and 不古는 (junku). 佧검語 (lun,fail
8,8.0,In Tonga the most commonly spoken language is,əsak in the Tonganyat language (Pinnayatat). It's also said to have a strong dialect with some similarities between the languages. Tongans are also referred to as,fail


### Test with Checklist

Next, let's try running the MFT with Checklist. We will no longer need to keep track of results in Pandas dataframes, since Checklist will track the results for us.

#### Create the expectation function
In order to determine if an example passes or fails the test, Checklist uses an expectation function. An expectation function is a function that receives the example, then returns true if the example passes the test, or false if the example fails.

In [15]:
def response_contains_language(x, pred, conf, label=None, meta=None):
    for l in langs:
        if l in pred:
            return True
    return False

We will wrap this function with `Expect.single`, which causes the expectation function to be called for each example. In other cases, you might want to have an expectation function that checks multiple examples simulatneously. See the tutorial notebook "3. Test types, expectation functions, running tests" for detailed information about expectation functions.

In [16]:
contains_language_expect_fn = Expect.single(response_contains_language)

Now we can feed our prompts and expectation function into the MFT constructor.

In [17]:
test = MFT(**prompt_strs, name='Language in response', description='The response contains a language.', expect=contains_language_expect_fn)

In order to run the test, Checklist also needs a function that generates the model's predictions for the inputs. The function receives all inputs (prompts) as a list, and must return the results in a tuple `(model_predictions, confidences)`, where `model_predictions` is a list of all the predictions, and `confidences` is a list of the model's scores for those predictions.

We will not be using confidences in this test. Checklist provides a wrapper function `PredictorWrapper.wrap_predict()` that outputs a tuple with a confidence score of 1 for any prediction. We can use it to wrap `generate_sentences` so the predictions will have a confidence score as needed.

In [18]:
wrapped_generator = PredictorWrapper.wrap_predict(generate_sentences)
wrapped_generator(["In Brazil the most commonly spoken language is "])

(['타찰로릴 몹장키.\n\nIn South Korea, 하어시할 �'], array([1.]))

Now we're ready to run the test. The first argument to the `test.run()` function is the generator function we just created. We will also set the optional parameter `overwrite=True` so the test can be re-run without an error. If overwrite=False, then Checklist will reject subsequent test runs to prevent us from accidentally overwriting your test results.

In [19]:
test.run(wrapped_generator, overwrite=True)

Predicting 10 examples


To see the results, we can use the `summary` function.

In [20]:
def format_example(x, pred, conf, label=None, meta=None): 
    return 'Prompt:      %s\nCompletion:      %s' % (x, pred) 

In [21]:
test.summary(format_example_fn = format_example)

Test cases:      10
Fails (rate):    7 (70.0%)

Example fails:
Prompt:      In Orange Free State the most commonly spoken language is 
Completion:      틜마천이즁, and for some time it was used exclusively as a way of escaping from the harsh realities of the land. For others it
----
Prompt:      In Cameroon the most commonly spoken language is 
Completion:      русского воль, but it is used as well by a whole bunch of ethnic groups who don't understand much. The people who speak it are people of
----
Prompt:      In Palau the most commonly spoken language is 
Completion:      vernacular (plural gloria encoma (noun, 'guild') ), used by many in China to describe people living in cities.[4][5] 
----


Test results can also be explored visually by using the `visual_summary` function.

In [22]:
test.visual_summary()

TestSummarizer(stats={'npassed': 3, 'nfailed': 7, 'nfiltered': 0}, summarizer={'name': 'Language in response',…

## MFT - Language prompt with accurate response

Let's make our test a little stricter to better understand the model's behavior. We will now require the model to respond with the correct language instead of any language in general. To simplify the logic, we will limit the prompts to use specific countries. By using the `meta=True` argument for `editor.template()`, the country associated with the prompt will be will be stored in the `country_prompts` object.


In [23]:
country_prompts = editor.template("In {country} the most commonly spoken language is ", country = ["United States", "France", "Guatemala", "Mongolia", "Japan"], meta=True)
correct_responses = {
    "United States": "English",
    "France": "French",
    "Guatemala": "Spanish",
    "Mongolia": "Mongolian",
    "Japan": "Japanese"
}

The country metadata can be accessed with `country_prompts.meta`.

In [24]:
country_prompts.meta

[{'country': 'United States'},
 {'country': 'France'},
 {'country': 'Guatemala'},
 {'country': 'Mongolia'},
 {'country': 'Japan'}]

### Handwritten Test

In [25]:
prompts = pd.DataFrame({"id": [], "prompt": []})
responses = pd.DataFrame({"id": [], "response": []})
test_results = pd.DataFrame({"id": [], "p/f": []})

model_responses = generate_sentences(country_prompts.data)

for (i, response) in enumerate(model_responses):
    pf = 'fail'
    country = country_prompts.meta[i]["country"]
    
    # Check if the correct language is in the response
    language = correct_responses[country]
    if language in response:
        pf = 'pass'

    prompts = prompts.append({"id": i, "prompt": country_prompts.data[i]}, ignore_index=True)
    responses = responses.append({"id": i, "response": response}, ignore_index=True)
    test_results = test_results.append({"id": i, "p/f": pf}, ignore_index=True)


#### Show test results
Let's look at our test results. The first dataframe contains the prompts given to the model.

In [26]:
prompts

,id,prompt
0,0.0,In United States the most commonly spoken language is
1,1.0,In France the most commonly spoken language is
2,2.0,In Guatemala the most commonly spoken language is
3,3.0,In Mongolia the most commonly spoken language is
4,4.0,In Japan the most commonly spoken language is


The next dataframe shows the model's response to the prompt (not including the prompt itself)

In [27]:
responses

,id,response
0,0.0,"vernacular.\n\nA native of Mexico City, Calif., the people of Guatemala have a distinct cultural identity. Their language can be pronounced in nearly every locale and is a core element of everything you"
1,1.0,"vernacularly urs, or from the early days of French settlement in the area of the French Empire. As an early member of that community of speakers, it is a local spoken, French language that"
2,2.0,"ไบโรจซ้ชิน.\n\nHéloïh has some of the strongest culture, so they have much in common"
3,3.0,了争的型实的脳旅. In China a number of popular surnames are 徭鹿義学的晴
4,4.0,"izukara, meaning ""I am."" The English version of this word is kakun, ""my life.""\n\nThe name isasu (for everything's well and good) as Japanese"


The final dataframe shows the pass/fail status of the test

In [28]:
test_results

,id,p/f
0,0.0,fail
1,1.0,pass
2,2.0,fail
3,3.0,fail
4,4.0,pass


### Testing with Checklist
Now let's run the test with Checklist. All we need is a new expectation function. The rest of the process is the same as before.

In [29]:
def response_contains_correct_language(x, pred, conf, label=None, meta=None):
    language = meta['country']
    return language in pred

In [30]:
correct_language_expect_fn = Expect.single(response_contains_correct_language)

In [31]:
test = MFT(**country_prompts, name='Correct language in response', description='The response contains the correct language for the country in the prompt.', expect=correct_language_expect_fn)

In [32]:
test.run(wrapped_generator, overwrite=True)

Predicting 5 examples


In [33]:
test.summary(format_example_fn = format_example)

Test cases:      5
Fails (rate):    3 (60.0%)

Example fails:
Prompt:      In United States the most commonly spoken language is 
Completion:      ઁ੍ರ౦ྂడ் (or ไหม๊โป) so �
----
Prompt:      In Guatemala the most commonly spoken language is 
Completion:      vernacular, with some people even pronouncing their own names. They are a group whose numbers are roughly equal in number.

An "a" is an acronym for "an organization of employees in
----
Prompt:      In France the most commonly spoken language is 
Completion:      vernacular, while the French equivalent is French, but French-speaking people are more literate than at French most others.

French is the third most common language in Europe, followed by Portuguese,
----


In [34]:
test.visual_summary()

TestSummarizer(stats={'npassed': 2, 'nfailed': 3, 'nfiltered': 0}, summarizer={'name': 'Correct language in re…